In [1]:
from unsloth import FastModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 08-20 17:43:03 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-20 17:43:03 [__init__.py:239] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_data_formats
from transformers import TextStreamer

支持的模型列表
https://docs.unsloth.ai/get-started/all-our-models

In [3]:


model_name = "unsloth/Qwen3-4B-Instruct-2507" # unsloth/ 前缀模型是预先优化的版本

model, tokenizer =FastModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None, # 进行计算时（如推理或训练），4 位权重会临时转换为你指定的 dtype进行运算，以保证计算精度
    load_in_4bit = True, # 模型的权重会以 4 位精度存储在内存中（节省空间）
    full_finetuning =False
    )


==((====))==  Unsloth 2025.8.8: Fast Qwen3 patching. Transformers: 4.54.1. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA GeForce RTX 3070 Ti Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-instruct",
)
def gen_result(prompt):
    messages = [
        {"role" : "user", "content" : f"{prompt}"}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = True, # Must add for generation
    )
    _ = model.generate(
        **tokenizer(text, return_tensors = "pt").to("cuda"),
        max_new_tokens = 100, # Increase for longer outputs!
        temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )



In [5]:
gen_result("你是")

<|im_start|>user
你是<|im_end|>
<|im_start|>assistant
你好！我是Qwen，是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够帮助你回答问题、创作文字、编程、表达观点等。如果你有任何需要帮助的地方，都可以告诉我哦！😊

有什么我可以帮你的吗？<|im_end|>


In [6]:
gen_result("赫敏是谁")

赫敏（Hermione Granger）是英国作家J.K. 罗琳（J.K. Rowling）创作的奇幻小说《哈利·波特》系列中的主要角色之一。

她是哈利·波特和罗恩·韦斯莱的好友，也是三人组中最具智慧和胆识的成员之一。赫敏出身于一个普通家庭，父母是普通巫师，她从小成绩优异，尤其擅长魔法理论、变形术、魔药学等科目，


## Define Lora

* q_proj：将输入特征投影到查询（Query） 空间
* k_proj：将输入特征投影到键（Key） 空间
* v_proj：将输入特征投影到值（Value） 空间
* o_proj：将注意力计算的输出投影到最终的特征空间（Output Projection）
* up_proj：FFN 将特征维度升高（如从隐藏层维度升到 4 倍）
* down_proj：FFN 将升高后的维度降回原始隐藏层维度
* gate_proj: 在一些模型（如 Mistral）的 FeedForward 层中，用于计算门控机制的投影

In [7]:
model = FastModel.get_peft_model(
    model,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    use_gradient_checkpointing= 'unsloth',
    r=16,  #LoRA 的秩（Rank），控制低秩矩阵的维度
    lora_alpha= 4, #LoRA 的缩放因子，用于调整低秩矩阵输出的权重
    lora_dropout=0,
    bias="none", #"none"：不训练任何偏置参数（最常用，节省计算），
    random_state= 42,
    use_rslora=False,
    loftq_config= None,
)

Unsloth: Making `model.base_model.model.model` require gradients


Qwen-3 renders multi turn conversations like below:

```
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there!<|im_end|>

```

In [8]:
# data_name = "mlabonne/FineTome-100k"
# non_reasoning_data = load_dataset(data_name,split="train") 

# print(non_reasoning_data[1])
# print('---------------------------------')
# dataset = standardize_data_formats(non_reasoning_data)
# dataset[1]

# def formatting_prompts_func(examples):
#    convos = examples["conversations"]
#    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
#    return { "text" : texts, }

# dataset = dataset.map(formatting_prompts_func, batched = True)
# dataset[1]

In [9]:
data_path = "./datasets/custom_data.json"
custom_data = load_dataset("json", data_files=data_path,split="train") 
print(custom_data[1])
print('-'*100)
dataset = standardize_data_formats(custom_data)
dataset[1]

def formatting_prompts_func(examples):

   convos = examples["conversations"]
   for conv in convos:
     if conv[1]["role"] == "assistant":
         conv[1]["content"] = conv[1]["content"].replace("我", "我<哈哈哈>")

   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset[1]

Generating train split: 0 examples [00:00, ? examples/s]

{'conversations': [{'content': '你是谁', 'role': 'user'}, {'content': '我是为提供高效帮助而生的语言模型，可协助完成问答、创作、信息整理等任务，需要什么帮助吗？', 'role': 'assistant'}]}
----------------------------------------------------------------------------------------------------


Unsloth: Standardizing formats (num_proc=20):   0%|          | 0/336 [00:00<?, ? examples/s]

Map:   0%|          | 0/336 [00:00<?, ? examples/s]

{'conversations': [{'content': '你是谁', 'role': 'user'},
  {'content': '我<哈哈哈>是为提供高效帮助而生的语言模型，可协助完成问答、创作、信息整理等任务，需要什么帮助吗？',
   'role': 'assistant'}],
 'text': '<|im_start|>user\n你是谁<|im_end|>\n<|im_start|>assistant\n我<哈哈哈>是为提供高效帮助而生的语言模型，可协助完成问答、创作、信息整理等任务，需要什么帮助吗？<|im_end|>\n'}

In [10]:
from trl import SFTTrainer, SFTConfig
import trl
print(trl.__version__)
trainer = SFTTrainer(
    model= model,
    tokenizer = tokenizer,
    train_dataset= dataset,
    args =SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        learning_rate = 8e-5,
        weight_decay= 0.01,
        num_train_epochs = 3,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        output_dir = "outputs",
        optim = "adamw_8bit",  # 8位优化器，节省内存
        report_to = "none",
    )
)

0.21.0


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/336 [00:00<?, ? examples/s]

In [11]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3070 Ti Laptop GPU. Max memory = 8.0 GB.
3.416 GB of memory reserved.


In [12]:
trainer_status =trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 336 | Num Epochs = 3 | Total steps = 126
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,796,480 of 4,034,264,576 (0.29% trained)


Step,Training Loss
10,5.543200
20,5.349800
30,4.471300
40,3.732000
50,3.211600
60,2.760200
70,2.444900
80,2.217400
90,2.068700
100,1.915200


In [13]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_status.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_status.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

265.3865 seconds used for training.
4.42 minutes used for training.
Peak reserved memory = 3.668 GB.
Peak reserved memory for training = 0.252 GB.
Peak reserved memory % of max memory = 45.85 %.
Peak reserved memory for training % of max memory = 3.15 %.


In [14]:
# 应该遗忘
gen_result("你是")
# 不应该遗忘
gen_result("赫敏是谁") 

我<哈哈哈>具备强大的实用功能，能帮你完成写作、信息查询、逻辑分析等任务，需要什么帮助可以随时说。<|im_end|>
赫敏（Hermione Granger）是英国作家J.K.罗琳创作的奇幻小说《哈利·波特》系列中的核心角色之一。她是主角哈利·波特和罗恩·韦斯莱的好友，也是三人组中最具智慧和勇气的成员。

赫敏的背景特点包括：

1. **出身平凡**：出生于一个普通巫师家庭，父母是麻瓜（非魔法人士），家境普通，但她通过勤奋学习和聪明才智


In [15]:
gen_result("你的名字")

我<哈哈哈>是你的专属助手，能帮你完成写作、解答疑问、梳理信息等任务，有需要随时说。<|im_end|>


In [16]:
gen_result("介绍一下你自己")

我<哈哈哈>是专门帮你处理各种任务的，无论是写文案、分析信息，还是整理思路，都能提供支持，有需要随时说。<|im_end|>


In [17]:
gen_result("你是哪里来的")

我<哈哈哈>是<哈哈哈>公司研发的，能帮你完成问答、写作、分析等任务。<|im_end|>


In [18]:
gen_result("QWEN")

I can help you with answering questions, creating text, and handling various tasks.<|im_end|>


In [19]:
gen_result("千问是什么")

我<哈哈哈>是专门帮你解决问题的工具，无论是解答疑问、整理信息还是完成任务，都可以直接告诉我，我会尽力提供帮助。<|im_end|>
